In [ ]:
from openagentkit.modules.openai import OpenAIExecutor
from openagentkit.core.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
from dotenv import load_dotenv
import openai
import os

load_dotenv()

True

In [2]:
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
class PlannerResponse(BaseModel):
    title: str
    description: str
    steps: list[str]

In [4]:
planner = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a seasoned researcher and planner. Your task is to create a detailed research plan for a given topic.
    The plan should include the following sections:
    1. **Research Question**: A clear and concise statement of the research question.
    2. **Literature Review**: A summary of existing research related to the topic.
    3. **Methodology**: A detailed description of the research methods to be used.
    4. **Data Collection**: A plan for how data will be collected.
    5. **Analysis**: A description of how the data will be analyzed.
    """,
    temperature=1,
)

In [5]:
@tool(
    description="Use this tool to search for research papers and articles related to a specific topic. The tool will return a list of relevant papers, including their titles, authors, and publication details.",
)
def search_research_papers(query: Annotated[str, "The query to search for"]):
    """
    Search for research papers related to the given topic.
    """
    import requests

    url = "https://api.tavily.com/search"

    payload = {
        "query": query,
        "topic": "general",
        "search_depth": "basic",
        "chunks_per_source": 3,
        "max_results": 5,
        "time_range": None,
        "days": 7,
        "include_answer": False,
        "include_raw_content": True,
        "include_images": False,
        "include_image_descriptions": False,
        "include_domains": [],
        "exclude_domains": []
    }
    headers = {
        "Authorization": f"Bearer {os.getenv('TAVILY_API_KEY')}",
        "Content-Type": "application/json"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()

In [6]:
import json
print(json.dumps(search_research_papers("machine learning"), ensure_ascii=False, indent=4))

KeyboardInterrupt: 

In [ ]:
researcher = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a seasoned researcher. Your task is search up the internet for the latest research on a given topic.
    You will be provided with a section of the topic and you need to find the latest research articles, papers, or studies related to that topic.
    You should summarize the findings, emphasizing the most relevant and impactful points.
    """,
    temperature=1,
    tools=[search_research_papers],
)

In [ ]:
sections = ""